# the `BusyFlyerMixin`

In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

In [3]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

In [20]:
class BusyFlyerDevice(ophyd.Device):
    """
    support APS Fly Scans that are operated by a busy record
    """

    busy = ophyd.Component(ophyd.EpicsSignal, BUSY_PV)
    time = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    motor = ophyd.Component(MyWaveform, X_WAVE_PV)
    signal = ophyd.Component(MyWaveform, Y_WAVE_PV)

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        self.hook_kickoff()

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def hook_kickoff(self):
        def cb(value, old_value, **kwargs):
            if self.busy.value in (BusyStatus.done):
                self._completion_status._finished(success=True)
        self.busy.subscribe(cb)
        
        self.t0 = time.time()
        self.busy.put(BusyStatus.busy)

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()" + str(self._completion_status))
        return self._completion_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {self.name, self.hook_describe_collect()}

    def hook_describe_collect(self):
        schema = {}
        for item in (self.time, self.motor, self.signal):
            structure = dict(
                source = item.wave.pvname,
                dtype = "number",
                shape = (1,)
            )
            schema[item.name] = schema
            
        return {self.name: schema}

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        self._completion_status = None
        for i in range(self.time.number_read.value):
            data = {}
            timestamps = {}
            for item in (self.time, self.motor, self.signal):
                data[item.name] = item.wave.value[i]
                timestamps[item.name] = t
            data[self.time.name] -= self.t0   # for this example only
            d = dict(
                time=time.time(),
                data=data,
                timestamps=timestamps
            )
            yield d


In [21]:
bflyer = BusyFlyerDevice(name="bflyer")

In [25]:
RE.abort()

TransitionError: RunEngine is already idle.

In [23]:
RE(bluesky.plans.fly([bflyer]))

A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediately, hit Ctrl+C again in the next 10 seconds.
Deferred pause acknowledged. Continuing to checkpoint.
Pausing...


RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.
